In [6]:
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from langchain_community.retrievers import PineconeHybridSearchRetriever
import os
load_dotenv()

True

In [10]:
key = os.getenv("PINECONE_API_KEY")

index_name = "hybrid-search"

# initilaize client 
pc = Pinecone(api_key=key)

# create index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension=384,  # dimenstion of dense vector
        metric="dotproduct", # fo sparse values supported only for dotproduct
        spec = ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ),
    )


In [13]:
index = pc.Index(index_name)
# index

In [15]:
# vector embedding and sparse matrix
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-V2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-V2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [17]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aayushmaanhooda/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [27]:
sentences = [
    "In 2023, I visited Mebourne",
    "In 2024, I visited SilverStone",
    "In 2025, I visited Texas",
]

# this is we applying tfidf on these values
bm25_encoder.fit(sentences)

# store values to json file 
bm25_encoder.dump("bm25_values.json")

# load your bm25 object 
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 1500.11it/s]


In [28]:
retriver = PineconeHybridSearchRetriever(
    embeddings=embeddings,
    sparse_encoder=bm25_encoder,
    index=index
)


In [29]:
retriver

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-V2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x13f3ae5d0>, index=<pinecone.db_data.index.Index object at 0x111654050>)

In [30]:
retriver.add_texts(
    [
    "In 2023, I visited Mebourne",
    "In 2024, I visited SilverStone",
    "In 2025, I visited Texas",
]
)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:03<00:00,  3.98s/it]


In [32]:
retriver.invoke("what city did i visit recent?")

[Document(metadata={'score': 0.199969038}, page_content='In 2025, I visited Texas'),
 Document(metadata={'score': 0.188349947}, page_content='In 2024, I visited SilverStone'),
 Document(metadata={'score': 0.175285563}, page_content='In 2023, I visited Mebourne')]